# Test pipeline from IMCprocess

## Create Img_adata object from csv input files
It is noted that csv file should have columns 'X_position' and 'Y_position'. Since csv output from Imacyte do not have X and Y coordinates, we need to add them from histogram csv output

In [6]:
import numpy as np
import scanpy as sc
import pandas as pd
from matplotlib import pyplot as plt
import os, sys
sys.path.append(os.path.join('..'))

import IMCprocess.Img_anndata as utan
import IMCprocess.utils as ut

In [7]:
def add_XY(histocat_csv, Imacyte_csv):
    new_df_ls = []
    for raw, new in zip(histocat_csv, Imacyte_csv):
        raw_df = pd.read_csv(raw)
        new_df = pd.read_csv(new, index_col=0)
        new_df['X_position'] = np.array(raw_df['X_position'])
        new_df['Y_position'] = np.array(raw_df['Y_position'])
        new_df_ls.append(new_df)
    return new_df_ls

In [ ]:
histocat_csv = []